# Final Deployment Model Training - SCM Project

**Project:** Safety Compliance Monitoring (SCM)

**Date:** January 2026

**Objective:** Train final 17-class deployment model for real-time PPE detection

**Approach:**
- Train on high-quality data from Model Zoo auto-labeling + SH17 base dataset
- Use best practices from SH17 paper (70.9% mAP benchmark)
- Optimize for edge deployment (Hailo-8 on Raspberry Pi 5)
- Target: 65-75% mAP50 overall

**Hardware Target:**
- Hailo-8 AI Accelerator (26 TOPS)
- Multi-camera support (4 cameras simultaneously)
- Real-time inference: 12-15 FPS per camera

## 1. Environment Setup

In [ ]:
# Install dependencies
!pip install ultralytics roboflow opencv-python matplotlib seaborn pandas pillow pyyaml

import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

from ultralytics import YOLO
import torch
import yaml
import os
from pathlib import Path
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Set up directories
base_dir = Path.home() / 'optense' / 'scm'
dataset_dir = base_dir / 'datasets' / 'final_merged'
models_dir = base_dir / 'models' / 'deployment'
results_dir = base_dir / 'training_results' / 'deployment'

for dir_path in [models_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"✓ Base directory: {base_dir}")
print(f"✓ Dataset directory: {dataset_dir}")
print(f"✓ Models will be saved to: {models_dir}")
print(f"✓ Training results: {results_dir}")

## 2. Download Dataset from Roboflow

Download your final merged dataset (SH17 base + auto-labeled supplements) from Roboflow.

**Requirements:**
- Roboflow API key (get from: https://app.roboflow.com/settings/api)
- Your merged dataset project name
- Dataset version number

In [ ]:
# ============================================
# ROBOFLOW CONFIGURATION
# ============================================

ROBOFLOW_API_KEY = "Xp2CBnbQfijsteBmE2Op"  # Get from: https://app.roboflow.com/settings/api
WORKSPACE_NAME = "optense"       # Your Roboflow workspace name
PROJECT_NAME = "scm-v1"         # Your merged dataset project name
VERSION_NUMBER = 2                      # Dataset version number

print("Roboflow Configuration:")
print("=" * 70)
print(f"Workspace: {WORKSPACE_NAME}")
print(f"Project: {PROJECT_NAME}")
print(f"Version: {VERSION_NUMBER}")
print(f"API Key: {'*' * (len(ROBOFLOW_API_KEY) - 4) + ROBOFLOW_API_KEY[-4:] if len(ROBOFLOW_API_KEY) > 4 else '(not set)'}")
print("=" * 70)

In [ ]:
# Download dataset from Roboflow
from roboflow import Roboflow

if ROBOFLOW_API_KEY == "YOUR_API_KEY_HERE":
    print("⚠️  Please set your Roboflow API key in the cell above!")
    print("\nSteps:")
    print("1. Go to: https://app.roboflow.com/settings/api")
    print("2. Copy your Private API Key")
    print("3. Paste it in ROBOFLOW_API_KEY above")
    print("4. Update WORKSPACE_NAME, PROJECT_NAME, and VERSION_NUMBER")
    print("5. Run this cell again")
else:
    try:
        print("Downloading dataset from Roboflow...")
        print("This may take several minutes depending on dataset size.\n")
        
        # Initialize Roboflow
        rf = Roboflow(api_key=ROBOFLOW_API_KEY)
        print("✓ Roboflow initialized")
        
        # Get project
        project = rf.workspace(WORKSPACE_NAME).project(PROJECT_NAME)
        print(f"✓ Project '{PROJECT_NAME}' accessed")
        
        # Download dataset
        print(f"\nDownloading version {VERSION_NUMBER}...")
        dataset = project.version(VERSION_NUMBER).download(
            'yolov8',
            location=str(dataset_dir)
        )
        
        print("\n" + "=" * 70)
        print("✓ DOWNLOAD COMPLETE!")
        print("=" * 70)
        print(f"Dataset location: {dataset.location}")
        
        # Update dataset_dir to actual location
        dataset_dir = Path(dataset.location)
        data_yaml_path = dataset_dir / 'data.yaml'
        
        # Verify download
        if data_yaml_path.exists():
            with open(data_yaml_path, 'r') as f:
                config = yaml.safe_load(f)
            
            print(f"\n✓ Dataset verified:")
            print(f"  Classes: {config['nc']}")
            
            # Handle both list and dict formats for class names
            if isinstance(config['names'], list):
                class_names = config['names']
            else:
                class_names = list((config['names'] if isinstance(config['names'], list) else list(config['names'].values())))
            
            print(f"  Class names: {class_names}")
            
            # Count images
            train_images = list((dataset_dir / 'train' / 'images').glob('*'))
            val_images = list((dataset_dir / 'val' / 'images').glob('*'))
            
            print(f"\n  Train images: {len(train_images)}")
            print(f"  Val images: {len(val_images)}")
            print(f"  Total: {len(train_images) + len(val_images)}")
            
            print("\n" + "=" * 70)
            print("✓ Ready to proceed with training!")
            print("=" * 70)
        else:
            print("\n⚠️  Warning: data.yaml not found after download")
            print("   Please check the download location.")
    
    except ImportError:
        print("❌ Error: roboflow package not installed")
        print("   Run: !pip install roboflow")
    
    except Exception as e:
        print(f"\n❌ Error downloading dataset: {e}")
        print("\nTroubleshooting:")
        print("  1. Check API key is correct (should start with your workspace name)")
        print("  2. Verify workspace name matches your Roboflow workspace")
        print("  3. Confirm project name is correct (check in Roboflow)")
        print("  4. Verify version number exists (check project versions)")
        print("  5. Ensure you have access permissions to the project")

## 3. Dataset Inspection

In [ ]:
# Read dataset configuration
data_yaml_path = dataset_dir / 'data.yaml'

if not data_yaml_path.exists():
    print(f"❌ ERROR: data.yaml not found at {data_yaml_path}")
    print("Please ensure your merged dataset is ready!")
else:
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    print("Dataset Configuration:")
    print("=" * 50)
    print(f"Number of classes: {data_config['nc']}")
    print(f"\nClass names:")
    for idx, name in data_config['names'].items():
        print(f"  {idx:2d}: {name}")
    
    print(f"\nTrain path: {data_config['train']}")
    print(f"Val path: {data_config['val']}")
    if 'test' in data_config:
        print(f"Test path: {data_config['test']}")

In [ ]:
# Count images and annotations
def count_dataset_statistics(dataset_root, split='train'):
    """
    Count images and analyze class distribution.
    """
    
    images_dir = dataset_root / split / 'images'
    labels_dir = dataset_root / split / 'labels'
    
    # Count images
    image_files = list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.png'))
    
    # Count annotations per class
    class_counts = {}
    total_annotations = 0
    images_with_annotations = 0
    
    for img_file in image_files:
        label_file = labels_dir / f"{img_file.stem}.txt"
        
        if label_file.exists():
            with open(label_file, 'r') as f:
                lines = f.readlines()
            
            if lines:
                images_with_annotations += 1
                
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        class_counts[class_id] = class_counts.get(class_id, 0) + 1
                        total_annotations += 1
    
    return {
        'total_images': len(image_files),
        'images_with_annotations': images_with_annotations,
        'total_annotations': total_annotations,
        'class_counts': class_counts
    }

# Analyze train and val splits
print("\nDataset Statistics:")
print("=" * 50)

train_stats = count_dataset_statistics(dataset_dir, 'train')
val_stats = count_dataset_statistics(dataset_dir, 'val')

print(f"\nTrain split:")
print(f"  Total images: {train_stats['total_images']}")
print(f"  Images with annotations: {train_stats['images_with_annotations']}")
print(f"  Total annotations: {train_stats['total_annotations']}")
print(f"  Avg annotations per image: {train_stats['total_annotations'] / train_stats['images_with_annotations']:.2f}")

print(f"\nValidation split:")
print(f"  Total images: {val_stats['total_images']}")
print(f"  Images with annotations: {val_stats['images_with_annotations']}")
print(f"  Total annotations: {val_stats['total_annotations']}")
print(f"  Avg annotations per image: {val_stats['total_annotations'] / val_stats['images_with_annotations']:.2f}")

In [ ]:
# Visualize class distribution
def visualize_class_distribution(train_stats, val_stats, class_names):
    """
    Create visualization of class distribution.
    """
    
    # Prepare data
    classes = []
    train_counts = []
    val_counts = []
    
    # Handle both list and dict formats
    if isinstance(class_names, list):
        class_ids = range(len(class_names))
    else:
        class_ids = sorted(class_names.keys())
    
    for class_id in class_ids:
        class_name = class_names[class_id] if isinstance(class_names, (list, dict)) else str(class_id)
        classes.append(class_name)
        train_counts.append(train_stats['class_counts'].get(class_id, 0))
        val_counts.append(val_stats['class_counts'].get(class_id, 0))
    
    # Create DataFrame
    df = pd.DataFrame({
        'Class': classes,
        'Train': train_counts,
        'Val': val_counts,
        'Total': [t + v for t, v in zip(train_counts, val_counts)]
    })
    
    # Sort by total count
    df = df.sort_values('Total', ascending=False)
    
    # Create visualization
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot 1: Bar chart
    x = range(len(df))
    width = 0.35
    
    axes[0].bar([i - width/2 for i in x], df['Train'], width, label='Train', alpha=0.8)
    axes[0].bar([i + width/2 for i in x], df['Val'], width, label='Val', alpha=0.8)
    axes[0].set_xlabel('Class')
    axes[0].set_ylabel('Instance Count')
    axes[0].set_title('Class Distribution (Train vs Val)', fontsize=14, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(df['Class'], rotation=45, ha='right')
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Plot 2: Class imbalance ratio
    max_count = df['Total'].max()
    df['Imbalance_Ratio'] = max_count / df['Total']
    
    colors = ['green' if r < 2 else 'orange' if r < 5 else 'red' for r in df['Imbalance_Ratio']]
    axes[1].barh(df['Class'], df['Imbalance_Ratio'], color=colors, alpha=0.7)
    axes[1].set_xlabel('Imbalance Ratio (max_count / class_count)')
    axes[1].set_ylabel('Class')
    axes[1].set_title('Class Imbalance Analysis', fontsize=14, fontweight='bold')
    axes[1].axvline(x=2, color='green', linestyle='--', label='Balanced (< 2:1)', alpha=0.5)
    axes[1].axvline(x=5, color='orange', linestyle='--', label='Moderate (< 5:1)', alpha=0.5)
    axes[1].axvline(x=10, color='red', linestyle='--', label='Severe (> 10:1)', alpha=0.5)
    axes[1].legend()
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(results_dir / 'dataset_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print summary table
    print("\nClass Distribution Summary:")
    print("=" * 70)
    print(df.to_string(index=False))
    print("\nImbalance Analysis:")
    print(f"  Most common class: {df.iloc[0]['Class']} ({df.iloc[0]['Total']} instances)")
    print(f"  Least common class: {df.iloc[-1]['Class']} ({df.iloc[-1]['Total']} instances)")
    print(f"  Max imbalance ratio: {df['Imbalance_Ratio'].max():.2f}:1")
    print(f"  Classes with severe imbalance (>10:1): {len(df[df['Imbalance_Ratio'] > 10])}")

# Visualize
visualize_class_distribution(train_stats, val_stats, data_config['names'])

## 3. Training Configuration

Based on best practices from:
- SH17 paper (70.9% mAP with YOLOv9-e)
- Model Zoo learnings (specialized models work well)
- Edge deployment requirements (Hailo-8 on Raspberry Pi 5)

In [ ]:
# ============================================
# TRAINING CONFIGURATION
# ============================================

# Model selection
MODEL_VARIANT = 'yolov8n'  # Options: yolov8n, yolov8s, yolov8m, yolov9c
# Note: yolov8n is fastest for edge deployment
#       yolov9c gives ~3-5% better accuracy but test Hailo compatibility first

# Training parameters (based on SH17 paper)
training_config = {
    # Dataset
    'data': str(data_yaml_path),
    
    # Model architecture
    'model': f'{MODEL_VARIANT}.pt',  # Use pretrained weights (transfer learning)
    
    # Training duration
    'epochs': 150,  # SH17 used 200, we use 150 (good balance)
    'patience': 30,  # Early stopping patience
    
    # Batch size (adjust based on GPU memory)
    'batch': 16,  # Increase if you have more VRAM (32, 64, 128)
    
    # Image size
    'imgsz': 640,  # SH17 validated this works well
    
    # Hardware
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 8,  # Data loading workers
    
    # Output
    'project': str(results_dir),
    'name': f'deployment_{MODEL_VARIANT}_v1',
    'exist_ok': True,
    'save': True,
    'save_period': 10,  # Save checkpoint every 10 epochs
    
    # Optimization
    'optimizer': 'AdamW',  # Better than SGD for most cases
    'lr0': 0.01,  # Initial learning rate
    'lrf': 0.01,  # Final learning rate (fraction of lr0)
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    
    # Loss weights
    'box': 7.5,  # Box loss weight
    'cls': 0.5,  # Classification loss weight
    'dfl': 1.5,  # Distribution focal loss weight
    
    # Data Augmentation (SH17 paper validated)
    'hsv_h': 0.015,  # HSV-Hue augmentation
    'hsv_s': 0.7,    # HSV-Saturation augmentation
    'hsv_v': 0.4,    # HSV-Value augmentation
    'degrees': 0.0,  # Rotation (already done in Roboflow)
    'translate': 0.1,  # Translation
    'scale': 0.5,    # Scale augmentation
    'shear': 0.0,    # Shear
    'perspective': 0.0,  # Perspective
    'flipud': 0.0,   # Vertical flip (not useful for overhead cameras)
    'fliplr': 0.5,   # Horizontal flip (SH17 used this)
    'mosaic': 1.0,   # Mosaic augmentation (SH17 used this)
    'mixup': 0.0,    # MixUp augmentation
    'copy_paste': 0.0,  # Copy-paste augmentation
    
    # Learning rate schedule
    'cos_lr': True,  # Cosine learning rate scheduler
    'close_mosaic': 10,  # Disable mosaic in last N epochs
    
    # Validation
    'val': True,
    'plots': True,  # Generate training plots
    
    # Advanced
    'resume': False,  # Resume from checkpoint
    'amp': True,  # Automatic Mixed Precision (faster training)
    'fraction': 1.0,  # Train on fraction of dataset (1.0 = all)
    'verbose': True,
    'seed': 42,  # Random seed for reproducibility
    'deterministic': True,  # Deterministic mode
}

print("Training Configuration:")
print("=" * 70)
print(f"Model: {training_config['model']}")
print(f"Epochs: {training_config['epochs']}")
print(f"Batch size: {training_config['batch']}")
print(f"Image size: {training_config['imgsz']}")
print(f"Device: {training_config['device']}")
print(f"Optimizer: {training_config['optimizer']}")
print(f"Learning rate: {training_config['lr0']} → {training_config['lr0'] * training_config['lrf']}")
print(f"\nAugmentation:")
print(f"  Mosaic: {training_config['mosaic']}")
print(f"  Horizontal flip: {training_config['fliplr']}")
print(f"  HSV: H={training_config['hsv_h']}, S={training_config['hsv_s']}, V={training_config['hsv_v']}")
print(f"\nOutput: {training_config['project']}/{training_config['name']}")
print("=" * 70)

## 4. Training Execution

**Expected Training Time:**
- YOLOv8n: 4-6 hours (150 epochs)
- YOLOv8s: 6-8 hours (150 epochs)
- YOLOv9c: 8-12 hours (150 epochs)

**Progress Monitoring:**
- Training metrics displayed in real-time
- Checkpoints saved every 10 epochs
- Best model saved automatically (based on validation mAP)

**What to Expect:**
- Target mAP50: 65-75% (based on clean data from Model Zoo)
- Person, face, head, hand: 80-85% mAP (high-frequency classes)
- Hardhat, safetyvest, shoes: 65-75% mAP (medium-frequency)
- Gloves, facemask: 50-60% mAP (minority classes)
- Glasses: 40-50% mAP (tiny objects)

In [ ]:
# Initialize model
print(f"Loading pretrained model: {training_config['model']}")
model = YOLO(training_config['model'])

print(f"\n✓ Model loaded successfully")
print(f"Parameters: {sum(p.numel() for p in model.model.parameters())/1e6:.2f}M")
print(f"Pretrained on: MS-COCO dataset")
print(f"Transfer learning: Enabled ✓")

In [ ]:
# Start training
print("\n" + "=" * 70)
print("STARTING TRAINING")
print("=" * 70)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Expected duration: 4-6 hours (for YOLOv8n)")
print("\nYou can monitor progress in real-time below.")
print("Training can be safely interrupted (Ctrl+C) and resumed later.")
print("=" * 70)
print("")

# Train the model
try:
    start_time = datetime.now()
    
    results = model.train(**training_config)
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds() / 3600  # Hours
    
    print("\n" + "=" * 70)
    print("TRAINING COMPLETE!")
    print("=" * 70)
    print(f"End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total duration: {duration:.2f} hours")
    print(f"\nBest weights saved to:")
    print(f"  {results_dir / training_config['name'] / 'weights' / 'best.pt'}")
    print("=" * 70)
    
except KeyboardInterrupt:
    print("\n\n⚠️  Training interrupted by user!")
    print("Checkpoint saved. You can resume training by setting 'resume': True in config.")
    
except Exception as e:
    print(f"\n\n❌ Training failed with error: {e}")
    import traceback
    traceback.print_exc()

## 5. Results Analysis

In [ ]:
# Load and display training results
results_path = results_dir / training_config['name']
results_csv = results_path / 'results.csv'

if results_csv.exists():
    # Read results
    results_df = pd.read_csv(results_csv)
    results_df.columns = results_df.columns.str.strip()  # Remove whitespace
    
    print("Training Results Summary:")
    print("=" * 70)
    
    # Get final metrics
    final_metrics = results_df.iloc[-1]
    
    print(f"\nFinal Epoch: {int(final_metrics['epoch'])}")
    print(f"\nValidation Metrics:")
    print(f"  mAP50:     {final_metrics['metrics/mAP50(B)']:.4f}")
    print(f"  mAP50-95:  {final_metrics['metrics/mAP50-95(B)']:.4f}")
    print(f"  Precision: {final_metrics['metrics/precision(B)']:.4f}")
    print(f"  Recall:    {final_metrics['metrics/recall(B)']:.4f}")
    
    print(f"\nTraining Losses (final):")
    print(f"  Box loss:  {final_metrics['train/box_loss']:.4f}")
    print(f"  Cls loss:  {final_metrics['train/cls_loss']:.4f}")
    print(f"  DFL loss:  {final_metrics['train/dfl_loss']:.4f}")
    
    # Find best epoch
    best_epoch = results_df['metrics/mAP50(B)'].idxmax()
    best_metrics = results_df.iloc[best_epoch]
    
    print(f"\nBest Epoch: {int(best_metrics['epoch'])}")
    print(f"  mAP50:     {best_metrics['metrics/mAP50(B)']:.4f}")
    print(f"  mAP50-95:  {best_metrics['metrics/mAP50-95(B)']:.4f}")
    print(f"  Precision: {best_metrics['metrics/precision(B)']:.4f}")
    print(f"  Recall:    {best_metrics['metrics/recall(B)']:.4f}")
    
    print("\n" + "=" * 70)
else:
    print("❌ Results file not found. Training may not have completed.")

In [ ]:
# Visualize training curves
if results_csv.exists():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: mAP curves
    axes[0, 0].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP50', linewidth=2)
    axes[0, 0].plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP50-95', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('mAP')
    axes[0, 0].set_title('Validation mAP', fontsize=12, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Plot 2: Precision & Recall
    axes[0, 1].plot(results_df['epoch'], results_df['metrics/precision(B)'], label='Precision', linewidth=2)
    axes[0, 1].plot(results_df['epoch'], results_df['metrics/recall(B)'], label='Recall', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Value')
    axes[0, 1].set_title('Precision & Recall', fontsize=12, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # Plot 3: Training losses
    axes[1, 0].plot(results_df['epoch'], results_df['train/box_loss'], label='Box Loss', linewidth=2)
    axes[1, 0].plot(results_df['epoch'], results_df['train/cls_loss'], label='Cls Loss', linewidth=2)
    axes[1, 0].plot(results_df['epoch'], results_df['train/dfl_loss'], label='DFL Loss', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].set_title('Training Losses', fontsize=12, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Plot 4: Validation losses
    axes[1, 1].plot(results_df['epoch'], results_df['val/box_loss'], label='Box Loss', linewidth=2)
    axes[1, 1].plot(results_df['epoch'], results_df['val/cls_loss'], label='Cls Loss', linewidth=2)
    axes[1, 1].plot(results_df['epoch'], results_df['val/dfl_loss'], label='DFL Loss', linewidth=2)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].set_title('Validation Losses', fontsize=12, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(results_dir / 'training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Training curves saved to: {results_dir / 'training_curves.png'}")

## 6. Model Validation & Per-Class Performance

In [ ]:
# Load best model for detailed validation
best_model_path = results_path / 'weights' / 'best.pt'

if best_model_path.exists():
    print(f"Loading best model from: {best_model_path}")
    best_model = YOLO(best_model_path)
    
    # Validate on test set
    print("\nRunning detailed validation...")
    val_results = best_model.val(
        data=str(data_yaml_path),
        batch=16,
        imgsz=640,
        device=0 if torch.cuda.is_available() else 'cpu',
        plots=True,
        save_json=True,
        verbose=True
    )
    
    print("\n✓ Validation complete!")
else:
    print("❌ Best model not found. Training may not have completed successfully.")

In [ ]:
# Analyze per-class performance
if best_model_path.exists():
    # Extract per-class metrics (from confusion matrix and validation results)
    print("\nPer-Class Performance Analysis:")
    print("=" * 70)
    
    # Check if confusion matrix exists
    confusion_matrix_path = results_path / 'confusion_matrix.png'
    if confusion_matrix_path.exists():
        print(f"\n✓ Confusion matrix saved to: {confusion_matrix_path}")
        print("  Review this to identify which classes are being confused.")
    
    # Check if per-class results exist
    results_json_path = results_path / 'predictions.json'
    if results_json_path.exists():
        print(f"\n✓ Detailed predictions saved to: {results_json_path}")
    
    print("\n📊 Key Performance Indicators:")
    print(f"  Overall mAP50: {val_results.box.map50:.4f}")
    print(f"  Overall mAP50-95: {val_results.box.map:.4f}")
    print(f"  Overall Precision: {val_results.box.mp:.4f}")
    print(f"  Overall Recall: {val_results.box.mr:.4f}")
    
    # Per-class mAP (if available)
    if hasattr(val_results.box, 'maps'):
        print("\n📋 Per-Class mAP50:")
        for idx, (class_name, map_value) in enumerate(zip((data_config['names'] if isinstance(data_config['names'], list) else list(data_config['names'].values())), val_results.box.maps)):
            status = "✓" if map_value > 0.7 else "⚠️" if map_value > 0.5 else "❌"
            print(f"  {status} {class_name:15s}: {map_value:.4f}")

## 7. Export Model for Deployment

In [ ]:
# Export to ONNX (required for Hailo conversion)
if best_model_path.exists():
    print("Exporting model to ONNX format...")
    print("This is required for Hailo HEF conversion.")
    print("")
    
    try:
        # Export to ONNX
        onnx_path = best_model.export(
            format='onnx',
            imgsz=640,
            simplify=True,
            opset=11,  # Hailo-compatible opset
            dynamic=False,  # Static shapes for Hailo
        )
        
        print(f"\n✓ ONNX export successful!")
        print(f"  Saved to: {onnx_path}")
        print(f"\n📋 Next steps:")
        print(f"  1. Transfer ONNX file to machine with Hailo Dataflow Compiler")
        print(f"  2. Convert to HEF using Hailo SDK:")
        print(f"     hailo parser onnx {onnx_path} --hw-arch hailo8")
        print(f"  3. Deploy HEF to Raspberry Pi + Hailo-8")
        
        # Copy to models directory
        import shutil
        final_onnx_path = models_dir / f'deployment_{MODEL_VARIANT}_best.onnx'
        shutil.copy(onnx_path, final_onnx_path)
        print(f"\n✓ ONNX copied to: {final_onnx_path}")
        
    except Exception as e:
        print(f"\n❌ ONNX export failed: {e}")
        print("You may need to export manually using the Ultralytics CLI.")

## 8. Model Summary & Deployment Checklist

In [ ]:
# Create model summary
if best_model_path.exists():
    summary = {
        'model': {
            'variant': MODEL_VARIANT,
            'architecture': 'YOLOv8' if 'yolov8' in MODEL_VARIANT else 'YOLOv9',
            'parameters': f"{sum(p.numel() for p in best_model.model.parameters())/1e6:.2f}M",
            'input_size': '640×640',
        },
        'training': {
            'epochs_trained': int(final_metrics['epoch']),
            'best_epoch': int(best_metrics['epoch']),
            'duration_hours': duration,
            'dataset_size': train_stats['total_images'] + val_stats['total_images'],
        },
        'performance': {
            'mAP50': float(val_results.box.map50),
            'mAP50-95': float(val_results.box.map),
            'precision': float(val_results.box.mp),
            'recall': float(val_results.box.mr),
        },
        'deployment': {
            'pytorch_model': str(best_model_path),
            'onnx_model': str(final_onnx_path) if 'final_onnx_path' in locals() else 'Not exported',
            'target_hardware': 'Raspberry Pi 5 + Hailo-8',
            'expected_fps': '12-15 FPS per camera (4 cameras)',
        },
        'metadata': {
            'training_date': datetime.now().isoformat(),
            'data_yaml': str(data_yaml_path),
            'classes': list((data_config['names'] if isinstance(data_config['names'], list) else list(data_config['names'].values()))),
            'num_classes': data_config['nc'],
        }
    }
    
    # Save summary
    summary_path = models_dir / f'deployment_{MODEL_VARIANT}_summary.json'
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print("\n" + "=" * 70)
    print("MODEL SUMMARY")
    print("=" * 70)
    print(json.dumps(summary, indent=2))
    print("\n" + "=" * 70)
    print(f"✓ Summary saved to: {summary_path}")
    print("=" * 70)

In [ ]:
# Deployment checklist
print("\n" + "=" * 70)
print("DEPLOYMENT CHECKLIST")
print("=" * 70)

checklist = [
    ("Training completed", best_model_path.exists()),
    ("mAP50 > 0.65 (target)", val_results.box.map50 > 0.65 if 'val_results' in locals() else False),
    ("ONNX export successful", 'final_onnx_path' in locals()),
    ("Model summary saved", summary_path.exists() if 'summary_path' in locals() else False),
]

for task, completed in checklist:
    status = "✅" if completed else "❌"
    print(f"{status} {task}")

print("\n📋 Next Steps:")
print("1. Convert ONNX to HEF using Hailo Dataflow Compiler")
print("2. Test HEF inference on Raspberry Pi + Hailo-8")
print("3. Integrate into SCM inference pipeline")
print("4. Multi-camera testing (4 streams)")
print("5. End-to-end validation with Rust runtime")
print("6. Deploy to production (Unilever + Pelindo)")
print("\n" + "=" * 70)

## 9. Performance Comparison with Expectations

In [ ]:
# Compare with expected performance
if 'val_results' in locals():
    expected_performance = {
        'Overall': {
            'target': 0.70,
            'minimum': 0.65,
            'actual': float(val_results.box.map50)
        },
    }
    
    print("\nPerformance Evaluation:")
    print("=" * 70)
    
    actual_map = expected_performance['Overall']['actual']
    target_map = expected_performance['Overall']['target']
    minimum_map = expected_performance['Overall']['minimum']
    
    print(f"\nOverall mAP50:")
    print(f"  Target:  {target_map:.2%} (SH17 paper benchmark)")
    print(f"  Minimum: {minimum_map:.2%} (acceptable)")
    print(f"  Actual:  {actual_map:.2%}", end="")
    
    if actual_map >= target_map:
        print(" ✅ EXCEEDS TARGET!")
        print("\n🎉 Excellent! Model performance exceeds expectations.")
    elif actual_map >= minimum_map:
        print(" ✓ MEETS REQUIREMENTS")
        print("\n✓ Good! Model performance is acceptable for deployment.")
    else:
        print(" ⚠️ BELOW MINIMUM")
        print("\n⚠️ Warning: Model performance below minimum threshold.")
        print("Consider:")
        print("  - Training for more epochs")
        print("  - Collecting more training data")
        print("  - Using larger model variant (yolov8s or yolov9c)")
        print("  - Reviewing data quality")
    
    print("\n" + "=" * 70)

## 10. Final Notes & Recommendations

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                   DEPLOYMENT MODEL TRAINING COMPLETE                 ║
╚══════════════════════════════════════════════════════════════════════╝

📊 Training Results:
   ✓ Model trained successfully
   ✓ Validation metrics computed
   ✓ ONNX export completed
   ✓ Model summary saved

🎯 Next Steps for Deployment:

   1. HEF Conversion (Hailo Dataflow Compiler)
      └─ Convert ONNX → HEF for Hailo-8 deployment
   
   2. Edge Device Testing
      └─ Test HEF inference on Raspberry Pi 5 + Hailo-8
   
   3. Pipeline Integration
      └─ Integrate model into SCM inference engine
   
   4. Multi-Camera Validation
      └─ Test with 4 RTSP camera streams simultaneously
   
   5. End-to-End Testing
      └─ Validate full system: Detection → Rust → iOS
   
   6. Production Deployment
      └─ Deploy to Unilever + Pelindo facilities

📝 Important Files:
""")

if best_model_path.exists():
    print(f"   PyTorch Model:  {best_model_path}")
if 'final_onnx_path' in locals():
    print(f"   ONNX Model:     {final_onnx_path}")
if 'summary_path' in locals():
    print(f"   Model Summary:  {summary_path}")
print(f"   Training Logs:  {results_path}")

print("""
💡 Performance Tips:
   • Model performance may improve with real-world data flywheel
   • Monitor false positive/negative rates in production
   • Collect edge cases for model v2 training
   • Consider per-facility fine-tuning if needed

🚀 You're ready to proceed to deployment!
""")

print("=" * 70)